In [1]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import databricks.koalas as ks

In [2]:
uri_db = 'mongodb+srv://<usernama>:<password>@bigdata.toqh2.mongodb.net'
spark_connector_uri = 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.7'

In [3]:
# Create a SparkSession object.
session = SparkSession.builder \
    .master('local') \
    .config('spark.mongodb.input.uri', uri_db) \
    .config('spark.jars.packages', spark_connector_uri) \
    .getOrCreate()

# Get context from SparkSession object.
context = session.sparkContext

In [4]:
# Read data from MongoDB and return two DataFrame objects, one
# for each collection contained in database.
df_reviews = session.read \
    .format('com.mongodb.spark.sql.DefaultSource') \
    .option('database', 'test') \
    .option('collection', 'reviews') \
    .load()
df_meta = session.read \
    .format('com.mongodb.spark.sql.DefaultSource') \
    .option('database', 'test') \
    .option('collection', 'meta') \
    .load()

# Drop MongoDB _id column in order to avoid error at runtime.
df_reviews = df_reviews.drop('_id')
df_meta = df_meta.drop('_id')

# Print collections schemas.
df_reviews.printSchema()
df_meta.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: integer (nullable = true)

root
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)


In [5]:
# Create two Koalas DataFrame from the Spark DataFrame objects.
kdf_reviews = ks.DataFrame(df_reviews)
kdf_meta = ks.DataFrame(df_meta)

# Extract sports and outdoors data from salesRank struct.
array_ranks = df_meta.select('salesRank.Sports &amp; Outdoors').to_koalas()

# Allow merge from different DataFrame objects.
ks.set_option('compute.ops_on_diff_frames', True)

# Assign a new column with the array_ranks data extracted above.
kdf_meta['sales_rank_sports_etc'] = array_ranks

# Compute join on asin attribute.
kdf_merge = kdf_reviews.merge(kdf_meta, on='asin')

In [6]:
# Query #1
kdf_1 = kdf_reviews \
    .groupby('asin') \
    .size().alias('reviews_count_product') \
    .sort_values(ascending=False) \
    .reset_index()

#kdf_1.head(100).to_csv(path='./to_csv/kdf_1.csv', num_files=1)

print("# I 100 prodotti con il maggior numero di recensioni #")
kdf_1.iloc[:100]

# I 100 prodotti con il maggior numero di recensioni #


,asin,reviews_count_product
0,B0000533G7,2239
1,7245456313,2033
2,B00004YVAJ,1503
3,B00005MOYK,1500
4,B00005UOSC,560
5,B000067Q0P,459
6,B0000644AF,401
7,B00005RCQS,381
8,B00004SQM9,376
9,B000067QMK,357


In [7]:
# Query #2
kdf_2 = kdf_reviews \
    .groupby('reviewerID') \
    .size().alias('reviews_count_reviewer') \
    .sort_values(ascending=False) \
    .reset_index()

#kdf_2.head(100).to_csv(path='./to_csv/kdf_2.csv', num_files=1)

print("# I 100 reviewer che hanno effettuato il maggior numero di recensioni #")
kdf_2.iloc[:100]

# I 100 reviewer che hanno effettuato il maggior numero di recensioni #


,reviewerID,reviews_count_reviewer
0,AF3EVH5OFWIQN,16
1,AVOGV98AYOFG2,14
2,AQUHJVX6O882V,12
3,AFP4UF1P3UX7U,9
4,A1M2T0J45TTE64,8
5,A1BG8QW55XHN6U,6
6,A31PDR78F1MNDI,5
7,A3T64QUILPVD2M,5
8,A2XGLU92QG2B6B,5
9,A281NPSIMI1C2R,4


In [8]:
# Query #3
kdf_3 =  kdf_merge \
    [kdf_merge.brand != ''] \
    .dropna(subset=['brand']) \
    .groupby('brand') \
    .size().alias('reviews_count_brand') \
    .sort_values(ascending=False) \
    .reset_index()

#kdf_3.head(50).to_csv(path='./to_csv/kdf_3.csv', num_files=1)

print("# Le 50 marche i cui prodotti sono stati maggiormente recensiti #")
kdf_3.iloc[:50]

# Le 50 marche i cui prodotti sono stati maggiormente recensiti #


,brand,reviews_count_brand
0,Victorinox,2566
1,Dr. Bronner,2239
2,Black Mountain,2189
3,Razor,1603
4,Coleman,933
5,Radio Flyer,634
6,Intex,593
7,Fisher-Price,560
8,Aerobie,521
9,Airzone,459


In [9]:
# Query #4
kdf_4 =  kdf_meta \
     .dropna(subset=['brand', 'price']) \
     .groupby('brand') \
     ['price'] \
     .mean().alias('price_mean') \
     .sort_values(ascending=False) \
     .reset_index()

#kdf_4.head(50).to_csv(path='./to_csv/kdf_4.csv', num_files=1)

print("# Le 50 marche i cui prodotti hanno un prezzo medio maggiore #")
kdf_4.iloc[:50]

# Le 50 marche i cui prodotti hanno un prezzo medio maggiore #


,brand,price_mean
0,Celestron,454.950000
1,Canon,399.890000
2,Title Boxing,317.120000
3,Viper,209.990000
4,Tomy,126.940000
5,MAPTECH PAPER CHARTS,116.950000
6,Bushnell,114.440000
7,Super Pogo,108.990000
8,Tripp Lite,107.450000
9,Suunto,105.696667


In [10]:
# Query #5
kdf_5 = kdf_reviews \
    .groupby('asin') \
    ['overall'] \
    .mean().alias('overall_mean_product') \
    .sort_values(ascending=False) \
    .reset_index()

kdf_5 = kdf_5 \
    .merge(kdf_1, on='asin') \
    .sort_values(by=['overall_mean_product', 'reviews_count_product'], ascending=False)

#kdf_5.head(100).to_csv(path='./to_csv/kdf_5.csv', num_files=1)

print("# I 100 prodotti con le migliori recensioni #")
kdf_5.iloc[:100]

# I 100 prodotti con le migliori recensioni #


,asin,overall_mean_product,reviews_count_product
148,0615302939,5.0,9
966,1300966955,5.0,9
515,B00000201L,5.0,8
20,B00006JK2H,5.0,7
302,1553602056,5.0,7
582,1304046176,5.0,7
2,724545647X,5.0,6
4,1300966947,5.0,6
293,1304801675,5.0,6
852,1300310065,5.0,6


In [11]:
# Query #6
kdf_6 = kdf_merge \
    [kdf_merge.brand != ''] \
    .dropna(subset=['brand']) \
    .groupby('brand') \
    ['overall'] \
    .mean().alias('overall_mean_brand') \
    .sort_values(ascending=False) \
    .reset_index()

kdf_6 = kdf_6 \
    .merge(kdf_3, on='brand') \
    .sort_values(by=['overall_mean_brand', 'reviews_count_brand'], ascending=False)

#kdf_6.head(100).to_csv(path='./to_csv/kdf_6.csv', num_files=1)

print("# Le 100 marche con le migliori recensioni #")
kdf_6.iloc[:100]

# Le 100 marche con le migliori recensioni #


,brand,overall_mean_brand,reviews_count_brand
143,NNG,5.000000,9
14,APBA International,5.000000,3
98,Titanium,5.000000,3
117,Football Fanatics,5.000000,2
154,MAPTECH PAPER CHARTS,5.000000,2
7,NBA Sportspicks,5.000000,1
10,3M,5.000000,1
25,BD&amp;A,5.000000,1
33,Logo Athletic,5.000000,1
68,Pavilion,5.000000,1


In [12]:
# Query #7 - #8
def get_helpful_rate(x):
    if x[1] == 0:
        return 0
    return (x[0]/x[1]) * 100

kdf_reviews['helpful_rate'] = kdf_reviews['helpful'].map(lambda x: get_helpful_rate(x))
kdf_reviews['helpful_pos'] = kdf_reviews['helpful'].map(lambda x: x[0])

kdf_reviews = kdf_reviews[kdf_reviews.helpful_pos != 0]

kdf_mean = kdf_reviews \
    [['reviewerID', 'helpful_rate']] \
    .groupby('reviewerID') \
    .mean() \
    .sort_values(by=['helpful_rate'], ascending=False) \
    .reset_index()

kdf_sum = kdf_reviews \
    [['reviewerID', 'helpful_pos']] \
    .groupby('reviewerID') \
    .sum() \
    .sort_values(by=['helpful_pos'], ascending=False) \
    .reset_index()

kdf_7_8 = kdf_mean \
    .merge(kdf_sum, on='reviewerID') \
    .sort_values(by=['helpful_rate', 'helpful_pos'], ascending=False)

In [13]:
# Query #7
kdf_7 = kdf_7_8.iloc[:100]

#kdf_7.to_csv(path='./to_csv/kdf_7.csv', num_files=1)

print("# I 100 reviewer che hanno effettuato recensioni con la maggiore utilità media #")
kdf_7

# I 100 reviewer che hanno effettuato recensioni con la maggiore utilità media #


,reviewerID,helpful_rate,helpful_pos
1169,A754P3SA07347,100.0,414
8231,AGVMN9XEO8WHE,100.0,155
7348,A1CIU06M0GZNHA,100.0,133
3838,AW33RDB2CET4O,100.0,127
3631,AU7O296O6PXZM,100.0,100
6338,A2LXYB1QS1GE2R,100.0,89
5473,A272YYWDRJJFXN,100.0,88
10566,A1C1NWYHIC0P5Q,100.0,84
8398,A66USRPEQF5EM,100.0,78
6851,A37TFIP0OMKGMW,100.0,76


In [19]:
# Query #8
kdf_8 = kdf_7_8 \
    .sort_values(by=['helpful_rate', 'helpful_pos'], ascending=True) \
    .iloc[:100]

#kdf_8.to_csv(path='./to_csv/kdf_8.csv', num_files=1)

print("# I 100 reviewer che hanno effettuato recensioni con la minore utilità media #")
kdf_8

# I 100 reviewer che hanno effettuato recensioni con la minore utilità media #


,reviewerID,helpful_rate,helpful_pos
7942,A137S47O4VFEN7,2.272727,1
3345,A3O2BDXR011HRB,2.564103,1
5620,A1G892XBOUQ6PW,2.702703,1
6248,AA3Q7A7JQUGQZ,3.030303,1
6175,A23NOJF66YG8NN,3.448276,1
745,A1EZT8O5KFP467,3.947368,3
1441,A22CH9DYXSW752,4.347826,1
10234,A43CAAZ5RMSFT,4.347826,1
5735,A1Y5DHBLW0C4S8,4.761905,1
10560,AZ1Z6EM4T09M8,4.878049,2


In [15]:
# Query #9
kdf_9 = kdf_meta \
    .dropna(subset=['sales_rank_sports_etc']) \
    .sort_values(by=['sales_rank_sports_etc'], ascending=True) \
    [['asin', 'sales_rank_sports_etc']]

#kdf_9.head(100).to_csv(path='./to_csv/kdf_9.csv', num_files=1)

print('# I 100 prodotti con il migliore ranking nelle vendite #')
kdf_9.iloc[:100]

# I 100 prodotti con il migliore ranking nelle vendite #


,asin,sales_rank_sports_etc
200,7245456313,15
530,B00004YVAJ,49
534,B00004YVB2,49
542,B00004YVB3,49
788,B00005UDHZ,88
789,B00005UDHY,88
850,B000067QMK,94
621,B00005BHO0,142
622,B00005BHNT,142
623,B00005BHNZ,142


In [21]:
# Query #10
kdf_10 = kdf_meta \
    [kdf_meta.brand != ''] \
    .dropna(subset=['brand', 'sales_rank_sports_etc']) \
    [['brand', 'sales_rank_sports_etc']] \
    .groupby('brand') \
    .mean() \
    .sort_values(by=['sales_rank_sports_etc'], ascending=True) \
    .reset_index()

#kdf_10.head(50).to_csv(path='./to_csv/kdf_10.csv', num_files=1)

print('# Le 50 marche i cui prodotti hanno il ranking medio migliore #')
kdf_10.iloc[:50]

# Le 50 marche i cui prodotti hanno il ranking medio migliore #


,brand,sales_rank_sports_etc
0,Mongoose,94.000000
1,Intex,404.000000
2,Wenzel,1109.000000
3,Razor,1375.500000
4,St. Pierre,2556.000000
5,Stanley,4952.000000
6,Spalding,7681.000000
7,Carrom,10603.000000
8,Black Mountain,10638.285714
9,Lifetime,11131.000000


In [17]:
kdf_result = kdf_meta.merge(kdf_5, on='asin')
cc1 = kdf_result[['price', 'overall_mean_product']].dropna().corr().iloc[0, 1]
print('# Correlazione tra il prezzo di un prodotto e il punteggio medio ottenuto nelle recensioni #')
print(cc1)

brand_categorical = {}
index_categorical = 1
def map_brand_to_categorical(brand):
    '''
        Map brand's name to categorical.

        Args:
            brand (string): brand's name.
        Return:
            int: categorical value.
    '''
    global brand_categorical
    global index_categorical

    if brand not in brand_categorical.keys():
        brand_categorical[brand] = index_categorical
        index_categorical += 1

    return brand_categorical[brand]

kdf_result['brand_cat'] = kdf_result['brand'].map(map_brand_to_categorical)
cc2 = kdf_result[['brand_cat', 'overall_mean_product']].dropna().corr().iloc[0, 1]
print('# Correlazione tra la marca di un prodotto e il punteggio medio ottenuto nelle recensioni #')
print(cc2)

cc3 = kdf_reviews[['helpful_rate', 'overall']].dropna().corr().iloc[0, 1]
print("# Correlazione tra l'utilità di una recensione e il punteggio assegnato dalla recensione al prodotto #")
print(cc3)

cc4 = kdf_reviews[['unixReviewTime', 'helpful_rate']].dropna().corr().iloc[0, 1]
print('# Correlazione tra la data di una recensione e l’utilità della stessa #')
print(cc4)

cc5 = kdf_reviews[['unixReviewTime', 'overall']].dropna().corr().iloc[0, 1]
print('# Correlazione tra la data di una recensione e il punteggio assegnato al prodotto #')
print(cc5)

kdf_result = kdf_meta.merge(kdf_5, on='asin')
cc6 = kdf_result[['sales_rank_sports_etc', 'overall_mean_product']].dropna().corr().iloc[0, 1]
print('# Correlazione tra il ranking delle vendite di un prodotto e i punteggi ottenuti nelle recensioni #')
print(cc6)

kdf_result = kdf_meta.merge(kdf_1, on='asin')
cc7 = kdf_result[['reviews_count_product', 'sales_rank_sports_etc']].dropna().corr().iloc[0, 1]
print('# Correlazione tra il numero delle recensioni di un prodotto e il ranking nelle vendite #')
print(cc7)

cc8 = kdf_result[['sales_rank_sports_etc', 'price']].dropna().corr().iloc[0, 1]
print('# Correlazione tra il ranking nelle vendite e il prezzo #')
print(cc8)

# Correlazione tra il prezzo di un prodotto e il punteggio medio ottenuto nelle recensioni #
-0.0041452296668180654
# Correlazione tra la marca di un prodotto e il punteggio medio ottenuto nelle recensioni #
-0.05554139686877985
# Correlazione tra l'utilità di una recensione e il punteggio assegnato dalla recensione al prodotto #
0.2897923346229214
# Correlazione tra la data di una recensione e l’utilità della stessa #
0.06549349762506135
# Correlazione tra la data di una recensione e il punteggio assegnato al prodotto #
-0.03819324100679914
# Correlazione tra il ranking delle vendite di un prodotto e i punteggi ottenuti nelle recensioni #
-0.1211785786118409
# Correlazione tra il numero delle recensioni di un prodotto e il ranking nelle vendite #
-0.16763087484741268
# Correlazione tra il ranking nelle vendite e il prezzo #
-0.07867217319046568


In [18]:
# kdf_merge['helpful_rate'] = kdf_reviews['helpful_rate']

# # Drop useless columns.
# kdf_subset = kdf_merge.drop(['helpful', 'reviewerID', 'reviewerName', \
#     'reviewTime', 'imUrl', 'related', 'salesRank', 'categories'], axis=1)

# # Drop null values on c ritical columns.
# kdf_subset = kdf_subset \
#     .dropna(subset=['helpful_rate', 'price', 'title', 'description', \
#         'brand', 'sales_rank_sports_etc'])

# # Join with kdf_1 in order to obtain reviews_count_product column.
# kdf_subset = kdf_subset.merge(kdf_1, on='asin')
# # Join with kdf_5 in order to obtain overall_mean_product column.
# # N.B. The review_count_product attribute is dropped because of
# # duplication.
# kdf_subset = kdf_subset.merge(kdf_5.drop('reviews_count_product'), on='asin')

# brand_categorical = {}
# index_categorical = 0
# def map_brand_to_categorical(brand):
#     '''
#         Map brand's name to categorical.

#         Args:
#             brand (string): brand's name.
#         Return:
#             int: categorical value.
#     '''
#     global brand_categorical
#     global index_categorical

#     if brand not in brand_categorical.keys():
#         brand_categorical[brand] = index_categorical
#         index_categorical += 1

#     return brand_categorical[brand]

# # Create new columns:
# # * brand_cat: brand's name in form of categorical;
# # * product_title_len: product's title length;
# # * product_description_len: product's description length;
# # * review_summary_len: review's title length;
# # * review_text_len: review's text length.
# kdf_subset['brand_cat'] = kdf_subset['brand'].map(map_brand_to_categorical)
# kdf_subset['product_description_len'] = kdf_subset['description'].map(len)
# kdf_subset['product_title_len'] = kdf_subset['title'].map(len)
# kdf_subset['review_text_sentiment_pos'] = kdf_subset['reviewText'].map( \
#     lambda x: sid.polarity_scores(x)['pos'])
# kdf_subset['review_summary_sentiment_pos'] = kdf_subset['summary'].map( \
#     lambda x: sid.polarity_scores(x)['pos'])

# # Drop non numeric column.
# kdf_subset = kdf_subset.drop(['asin', 'brand', 'description', 'title', \
#     'reviewText', 'summary'], axis=1)

# kdf_subset.to_csv(path='./to_csv/kdf_subset.csv', num_files=1)